In [ ]:
using Pkg
# environmentName will be used several times in this notebook
environmentName = "02-networks"
Pkg.activate(joinpath(dirname(pwd()), "conf", environmentName))
Pkg.instantiate();

# Network Analysis

What this notebook does:
1. load graph-data used in this notebook
2. create the adjacency matrix $A$
3. create a graph $G$ from $A$ (sparse, no loops) and plot the graph
5. calculate the Laplace matrix $L$
6. calculate the eigenvalues $\lambda_i$ of $L$ and plot the spectrum
7. find the largest component $H$ von $G$ and plot its graph
8. calculate the eigenvalues of $H$ and plot the spectrum
9. use the eigenvector $f_H$ of eigenvalue $\lambda_H$ to cluster $H$ (split $H$ in two clusters)
9. use the eigenvector $f_{|V(H)|-1}$ of eigenvalue $\lambda_{|V(H)|-1}$ (find "most important" nodes)

# Load graph data

The data for this notebook is located in `"data/02-networks/graph.json"`. This [JSON file](https://en.wikipedia.org/wiki/JSON) contains an array of entries, each with the attributes `"name"` and `"links"`. The `"links"` attribute is itself an array of strings. Each entry represents a [Wikipedia page (english)](https://en.wikipedia.org/wiki/Main_Page) whose name or title contains either *"data"* or *"science"* (case-insensitive). The pages (identified by `"name"`) serve as the **nodes** of our graph, and the `"links"` define **directed edges** between them.

However, in the analysis that follows, we will **disregard the directionality** of the edges and consider two nodes to be connected by an edge if a directed link exists between them in either direction.

## Example data (made up)
```JSON
[
    {
        "name": "Data_science",
        "links": [
            "Data",
            "Science"
        ]
    },
    {
        "name": "Data",
        "links": [
            "Data_science",
            "Science"
        ]
    },
    {
        "name": "Science",
        "links": [
            "Data_science",
            "Database"
        ]
    },
    {
        "name": "Database",
        "links": [
            "Data"
        ]
    }
]
```

## Example Graph

We consider a graph $G = (V, E)$ with:
$V = \{\text{Data\_science},\ \text{Data},\ \text{Science},\ \text{Database}\}$ and

$E = \{\{\text{Data\_science}, \text{Data}\},\ \{\text{Data\_science}, \text{Science}\},\ \{\text{Data}, \text{Science}\},\ \{\text{Database}, \text{Science}\},\ \{\text{Data}, \text{Database}\}\}$

For simplicity, the nodes $V$ can also be indexed numerically.

In [ ]:
using JSON3

data = JSON3.read(read(joinpath(dirname(pwd()), "data", environmentName, "graph.json")))
println("typeof(data) = " * string(typeof(data)))
println("Anzahl Einträge im Array = " * string(length(data)))
# print the first 5 entries
println(first(data, 10))

# Calculate Adjaceny matrix $A$

In [ ]:
using SparseArrays

n = length(data)
V = 1:n
# maps page names to node indices
name_map = Dict()
A = zeros(Int, n, n)
E = []

# associate page names with node indices
for i = V
    name_map[data[i].name] = i
end

for i = V
    page = data[i]
    for j = 1:length(page.links)
        linked_page = page.links[j]
        # ignore trailing links
        if haskey(name_map, linked_page)
            # A is a symmetric matrix (undirected graph)
            A[i, name_map[linked_page]] = A[name_map[linked_page], i] = 1
        end
    end
end

A = sparse(A)
A

The edges of graph $G$ are defined by $A$.

In [ ]:
using Graphs

G = Graphs.Graph(A)

We will remove all isolated nodes.

In [ ]:
d = Graphs.degree(G)
not0 = findall(d .!= 0)
V = V[not0]
A = A[not0,not0]
d = d[not0]
G = Graphs.Graph(A)

The following cell visualises the graph. Some nodes of the graph seem to have many neighbours of degree one which look like leafs in a treelike graph while other nodes are more interconnected.

In [ ]:
using GraphPlot, Plots, Compose, Cairo, Fontconfig

c = sortperm(d, rev = true)[1:5]
labels = ["" for v in V]
node_names = [data[v].name for v in V]
labels[c] = node_names[c]
layout = spring_layout(G; C=5)
P1 = gplot(G, layout...;
        linetype="curve", 
        nodelabel=labels,
        nodelabeldist=2,
        nodefillc = colorant"steelblue"
)
draw(PNG(joinpath(dirname(pwd()), "img", environmentName, "graph_visualisation.png"), 16cm, 16cm), P1)
P1

---
---
# The Laplace matrix

In [ ]:
n = length(d)
L = zeros(n, n)
for i in 1:n
    for j in 1:n
        if i == j
            L[i,j] = 1
        elseif A[i,j] == 1
            L[i,j] = - 1 / sqrt(d[i] * d[j])
        end
    end
end
L

The eigenvalues of $L$ are called the spectrum and its plot looks like this:

In [ ]:
using LinearAlgebra

E = eigen(L)
λ = E.values
P2 = plot(λ, label = "The spectrum of G", 
             legend = :bottomright,
             linewidth = 3,
             xlab = "i",
             ylab = "λᵢ")
savefig(P2, joinpath(dirname(pwd()), "img", environmentName, "spectrum_of_G.png"))
P2

The number of components of $G$ is equal to $\text{dim}(\text{ker}(L)) = \text{dim}(\text{Eig}_0)$.

In [ ]:
n_components = count(λ .< 1e-14)

In [ ]:
C = connected_components(G)

In [ ]:
sum(λ), n

---
# Analysing the largest component

In [ ]:
k = findmax(length.(C))
I = C[k[2]];
H = Graph(A[I, I])

In [ ]:
layout_H = (layout[1][I], layout[2][I])
P3 = gplot(H, layout_H...;
        linetype="curve", 
        nodelabeldist=2,
        nodefillc = colorant"steelblue"
)
draw(PNG(joinpath(dirname(pwd()), "img", environmentName, "graph_H_visualisation.png"), 16cm, 16cm), P3)
P3

In [ ]:
L_H = L[I,I]
E_H = eigen(L_H)
λ_H = E_H.values
P4 = plot(λ_H, legend = false, linewidth = 3)
savefig(P4, joinpath(dirname(pwd()), "img", environmentName, "spectrum_of_H.png"))
P4

Let $w$ be the eigenvector to eigenvalue $\lambda_G$. We are gonna plot the graph and color the nodes red if their corresponding entry in $w$ is positive, blue if the entry is negative, and yellow if the entry is 0. This induces a clustering into two partitions.

In [ ]:
using Images, IJulia

w = E_H.vectors[:,2]
cols = []
for wᵢ in w
    if abs(wᵢ) > 1e-12 
        if wᵢ < 0
            push!(cols, colorant"steelblue")
        else
            push!(cols, colorant"indianred")
        end
    else
        push!(cols, colorant"orange")
    end
end
layout= x -> spring_layout(x; C=5)
P5 = gplot(H, layout_H...;
        linetype="curve", 
        nodelabeldist=2,
        nodefillc = cols
)
# wir speichern das Bild und plotten dann das nachgeladene Bild,
# weil jupyter teilweise Probleme damit hat gefärbte Graphen anzuzeigen, die viele Knoten enthalten
imagePath = joinpath(dirname(pwd()), "img", environmentName, "clustering_of_H_by_smallest_lambda.png")
draw(PNG(imagePath, 16cm, 16cm), P5)
IJulia.display(load(imagePath))

Lets have a look at the node-labels of their respective clusters:

In [ ]:
[entry.name for entry in data[I][w .> 0]]

In [ ]:
[entry.name for entry in data[I][w .< 0]]

Similarly we can partition the node set by using the eigenvector to eigenvalue $\lambda_{|V(H)|-1}$.

In [ ]:
using IJulia, Images

u = E_H.vectors[:,end]
cols = []
for uᵢ in u
    if abs(uᵢ) > 1e-12 
        if uᵢ < 0
            push!(cols, colorant"steelblue")
        else
            push!(cols, colorant"indianred")
        end
    else
        push!(cols, colorant"orange")
    end
end
P6 = gplot(H, layout_H...;
        linetype="curve",
        nodefillc = cols
)
# wir speichern das Bild und plotten dann das nachgeladene Bild,
# weil jupyter teilweise Probleme damit hat gefärbte Graphen anzuzeigen, die viele Knoten enthalten
imagePath = joinpath(dirname(pwd()), "img", environmentName, "clustering_of_H_by_largest_lambda.png")
draw(PNG(imagePath, 16cm, 16cm), P6)
IJulia.display(load(imagePath))

In [ ]:
[entry.name for entry in data[I][u .> 1e-12]]

In [ ]:
[entry.name for entry in data[I][u .< 1e-12]]